In [9]:
import re
import jieba
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1.准备数据，建立vocab

In [2]:
df = pd.read_csv('cmn.txt', sep='\t', header=None, names=['en', 'zh'])

In [16]:
all_tokens = set()
for en in df['en']:
    for i in en.split(' '):
        all_tokens.add(i.replace('.', '').replace(',', '').replace('!', '').replace('?', '').replace(' ', ''))

In [25]:
class En2Zh(Dataset):
    def __init__(self):
        super().__init__()
        df = pd.read_csv('cmn.txt', sep='\t', header=None, names=['en', 'zh'])
        
        # 英文按空格切分，创建词表
        self.ens = []
        counter = Counter()
        for en in df['en']:
            tokenized = [x.replace('.', '').replace(',', '').replace('!', '').replace('?', '').replace(' ', '') for x in en.split(' ') if x not in ['.', '!', '']]
            counter.update(tokenized)
            self.ens.append(tokenized)
            
        self.en_vocab = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
        self.en_vocab.set_default_index(self.en_vocab['<unk>'])
        
        # 中文按字切分，创建词表
        counter = Counter()
        self.zhs = []
        for zh in df['zh']:
            zh_list = [x for x in zh if x not in ['.', ',', '。', '！', '？', '!', '?']]
            counter.update(zh_list)
            self.zhs.append(zh_list)
            
        self.zh_vocab = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
        self.zh_vocab.set_default_index(self.zh_vocab['<unk>'])
        
    def __len__(self):
        return len(self.zhs)
        
    def __getitem__(self, idx):
        en_encoded = [self.en_vocab['<bos>']] + [self.en_vocab[token] for token in self.ens[idx]] + [self.en_vocab['<eos>']]
        zh_encoded = [self.zh_vocab['<bos>']] + [self.zh_vocab[token] for token in self.zhs[idx]] + [self.zh_vocab['<eos>']]
        
        return torch.LongTensor(en_encoded), torch.LongTensor(zh_encoded)

# collate_fn，传给DataLoader，对于每一个batch，将其中的句子都pad成和最长的一样长，用PAD_IDX填充
def collate_fn(data_batch):
    en_batch, zh_batch = [], []
    for en_item, zh_item in data_batch:
        en_batch.append(en_item)
        zh_batch.append(zh_item)
    en_batch = pad_sequence(en_batch, padding_value=1, batch_first=True)
    zh_batch = pad_sequence(zh_batch, padding_value=1, batch_first=True)
    return en_batch, zh_batch

dataset = En2Zh()

In [32]:
dataset[0]

(tensor([2, 4, 3]), tensor([2, 4, 3]))

# 2.构建Encoder、Decoder和Seq2Seq

In [33]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, dropout=0.5):
        super().__init__()
        self.vocab_size = vocab_size  # encoder vocab size
        self.embed = nn.Embedding(vocab_size, embed_size)  # 将vocab size嵌入到embed size
        # GRU循环网络，输入[steps * batch_size * embde_size]，输出[steps * batch_size * hidden_size]
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers=2, batch_first=True)  
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 返回encoder的输出，大小为[steps*batch_size*hidden_size]
        # 返回encoder GRU隐层的最后一步
        embedded = self.dropout(self.embed(x))
        enc_output, enc_hidden = self.rnn(embedded)
        return enc_output, enc_hidden  


class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, dropout=0.5):
        super().__init__()
        self.vocab_size = vocab_size  # decoder vocab size
        self.embed = nn.Embedding(vocab_size, embed_size)  # 将vocab size嵌入到embed size
        # GRU循环网络，输入[steps*batch_size*embde_size]，输出[steps*batch_size*hidden_size]
        self.rnn = nn.GRU(embed_size, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)  # 全连接层，输出尺寸为decoder vocab size
        self.dropout = nn.Dropout(dropout)

    def forward(self, y, hidden):
        embedded = self.dropout(self.embed(y))
        dec_output, dec_hidden = self.rnn(embedded, hidden)
        dec_output = self.fc(dec_output)
        return dec_output, dec_hidden


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        enc_output, hidden = self.encoder(src)  # 首先拿到encoder的output和最后一个时间步的隐状态
        batch_size, max_len = tgt.shape[0], tgt.shape[1]  
        # Seq2Seq output尺寸为[批量大小 * 步长 * vocab_size]
        output = torch.zeros(batch_size, max_len, self.decoder.vocab_size).to(device)
        # 先拿tgt的第一个时间步，即<bos>开始，输入到decoder中，
        # 第一个时刻的hidden为encoder的最后一个时间步的hidden
        y_t = tgt[:, 0]  
        # 第二步开始，遍历tgt的每一个时间步，decoder输入为上一时刻的预测结果，以及上一时刻的hidden
        for t in range(1, max_len):  
            y_t.unsqueeze_(1)
            y_t, hidden = self.decoder(y_t, hidden)
            y_t.squeeze_(1)
            output[:, t, :] = y_t
            y_t = y_t.argmax(1)
        return output
    
# 初始化encoder、decoder和Seq2Seq
enc = Encoder(vocab_size=len(dataset.en_vocab), embed_size=512, hidden_size=512)
dec = Decoder(vocab_size=len(dataset.zh_vocab), embed_size=512, hidden_size=512)
model = Seq2Seq(enc, dec).to(device)
model

Seq2Seq(
  (encoder): Encoder(
    (embed): Embedding(7214, 512)
    (rnn): GRU(512, 512, num_layers=2, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embed): Embedding(3434, 512)
    (rnn): GRU(512, 512, num_layers=2, batch_first=True)
    (fc): Linear(in_features=512, out_features=3434, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

# 3.训练模型

In [49]:
dataloader = DataLoader(dataset, batch_size=512, shuffle=True, collate_fn=collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=1).to(device)

model.train()
for epoch in range(200):
    epoch_loss = 0
    for src, tgt in dataloader:
        src = src.to(device)
        tgt = tgt.to(device)
        
        output = model(src, tgt)
        output = output[:, 1:, :].reshape(-1, output.shape[-1])
        tgt = tgt[:, 1:].reshape(-1)
        loss = criterion(output, tgt)
        
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        epoch_loss += loss.item()
    print('epoch:', epoch + 1, ', loss:', format(epoch_loss / len(dataset), '.6f'))

epoch: 1 , loss: 0.000811
epoch: 2 , loss: 0.000746
epoch: 3 , loss: 0.000735
epoch: 4 , loss: 0.000724
epoch: 5 , loss: 0.000715
epoch: 6 , loss: 0.000714
epoch: 7 , loss: 0.000710
epoch: 8 , loss: 0.000703
epoch: 9 , loss: 0.000709
epoch: 10 , loss: 0.000711
epoch: 11 , loss: 0.000696
epoch: 12 , loss: 0.000707
epoch: 13 , loss: 0.000706
epoch: 14 , loss: 0.000697
epoch: 15 , loss: 0.000701
epoch: 16 , loss: 0.000696
epoch: 17 , loss: 0.000712
epoch: 18 , loss: 0.000684
epoch: 19 , loss: 0.000695
epoch: 20 , loss: 0.000693
epoch: 21 , loss: 0.000679
epoch: 22 , loss: 0.000682
epoch: 23 , loss: 0.000667
epoch: 24 , loss: 0.000656
epoch: 25 , loss: 0.000670
epoch: 26 , loss: 0.000661
epoch: 27 , loss: 0.000665
epoch: 28 , loss: 0.000656
epoch: 29 , loss: 0.000650
epoch: 30 , loss: 0.000654
epoch: 31 , loss: 0.000663
epoch: 32 , loss: 0.000663
epoch: 33 , loss: 0.000653
epoch: 34 , loss: 0.000654
epoch: 35 , loss: 0.000648
epoch: 36 , loss: 0.000654


KeyboardInterrupt: 

# 5.翻译

In [54]:
model.eval()
# 先讲中文输入到encoder中，拿到encoder的hidden，从<bos>依次输入到decoder中，
# 直到预测到<eos>停止，或者超过设定的max_len时停止
def translate(en, max_len=10):
    tokenized = [x.replace('.', '').replace(',', '').replace('!', '').replace('?', '').replace(' ', '') for x in en.split(' ') if x not in ['.', '!', '']]
    en_idx = [dataset.en_vocab['<bos>']] + dataset.en_vocab.lookup_indices(tokenized) + [dataset.zh_vocab['<eos>']]
    en_idx = torch.tensor(en_idx, dtype=torch.long, device=device).unsqueeze(0)
    
    zh_bos = dataset.zh_vocab['<bos>']
    enc_output, hidden = model.encoder(en_idx)
    preds = []
    y = torch.LongTensor([zh_bos]).to(device)
    
    for t in range(max_len):
        y.unsqueeze_(0)
        y, hidden = model.decoder(y, hidden)
        y.squeeze_(0)
        y = y.argmax(1)
        if y.item() == dataset.zh_vocab['<eos>']:
            break
        preds.append(dataset.zh_vocab.get_itos()[y.item()])
    return ''.join(preds)

In [58]:
df = pd.read_csv('cmn.txt', sep='\t', header=None, names=['en', 'zh'])

for i in df.sample(20).index:   
    print('英文：', df['en'][i])
    print('中文：', df['zh'][i])
    print('模型结果：', translate(df['en'][i]))
    print()

英文： She has a strong wish to work as an interpreter.
中文： 她非常想当口译。
模型结果： 她非常想译地

英文： We watch TV every day.
中文： 我们每天看电视。
模型结果： 我们隨天看天视

英文： There's nobody here.
中文： 這裡沒人。
模型结果： 沒什麼人

英文： Why am I still here?
中文： 为什么我还在这里？
模型结果： 为什么我还在这里

英文： Do you know the difference between a microscope and a telescope?
中文： 你知道显微镜和望远镜的差别吗？
模型结果： 你知道显微镜和望远镜

英文： He played tennis.
中文： 他打了网球。
模型结果： 网响彎球表

英文： They forgot to lock the door.
中文： 他们忘了锁门。
模型结果： 門被忘了了

英文： Will you please turn down the radio?
中文： 請你把收音機關小聲一點好嗎？
模型结果： 你你博你收會音機嗎

英文： How can you say that?
中文： 你怎麼能那麼說？
模型结果： 你怎麼多

英文： How long will you be staying?
中文： 你会待多长时间？
模型结果： 火子间多久

英文： Tom was late for dinner.
中文： 湯姆晚餐遲到了。
模型结果： 湯姆早上遲回來

英文： He occasionally visited me.
中文： 他偶尔会来拜访我。
模型结果： 舞拜着我

英文： I want to be the one who decides.
中文： 我想成为决策的人。
模型结果： 我想成成为策的

英文： Why are you still unmarried?
中文： 你為甚麼還不結婚？
模型结果： 为甚麼還不結束

英文： She glanced shyly at the young man.
中文： 她羞怯地看了一眼那個年輕人。
模型结果： 看當開了五些年輕女人

英文： It's time to talk.
中文： 到談話的時間了。
模型结果： 